# rocALUTION GPU solvers

### Description 
Test of different iterative linear solvers and preconditionners from library rocALUTION library https://github.com/ROCmSoftwarePlatform/rocALUTION. These solvers run either on GPU AMD card, either on CPU.

The test cases are run sequentially and parallel with TRUST on 4 cores. The size of the pressure linear system is 2 592 000 rows.

NB: They will be fully available in the TRUST 1.9.1 version.

In [ ]:
from trustutils import run
run.TRUST_parameters()
run.introduction('Pierre LEDAC')
# Creation des jeux de donnees
NP=4
seuil="seuil 1.e-4 impr"
cases=[
       ("gc"            ,"CG"             ,"gcp      { precond null   { } %s }" % seuil),
       ("gcp_jacobi"    ,"CG/Jacobi"      ,"gcp      { precond jacobi { } %s }" % seuil),
       ("bicgstab_multicoloredgs" ,"BiCGStab/MulticoloredGS","bicgstab { precond multicoloredgs { } %s }" % seuil),
       ("bicgstab_ilu0" ,"BiCGStab/ILU(0)","bicgstab { precond ilu    { level 0 } %s }" % seuil),
       ("bicgstab_pairwiseamg","BiCGStab/Pairwise-AMG","bicgstab { precond pairwiseamg  { } %s }" % seuil),
       ("bicgstab_uaamg","BiCGStab/UA-AMG","bicgstab { precond uaamg  { } %s }" % seuil),
       ("bicgstab_saamg","BiCGStab/SA-AMG","bicgstab { precond saamg  { } %s }" % seuil),
      ]
run.reset()
run.initCaseSuite()
for case,label,syntax in cases:
    # Create test case:
    run.executeCommand("cas=%s;mkdir -p $cas;cd $cas;cp ../base.data $cas.data;ln -s -f ../post_run ." % case, verbose=False)
    cas = run.addCase(case,"%s.data" % case)
    cas.substitute("_solveur_",syntax)
    # Create a parallel test case:
    run.executeCommand("cas=%s;cd $cas;make_PAR.data $cas %s;exit 0" % (case,NP), verbose=False)
    cas = run.addCase(case,"PAR_%s.data" % case, NP)
    
run.printCases()

In [ ]:
run.runCases()

# Convergence

In [ ]:
from trustutils import plot
    
a = plot.Graph("Relative residual ||Ax(it)-b||/||Ax(0)-b|| during the fist time step:","",1,1,[10,5])

for case,label,syntax in cases:
    cols = plot.loadText(case+"/%s.res" % case)
    a.add(cols[0],cols[1],label="%s" % label)

a.label("Iteration","Residual")
a.subplot.set_yscale('log')

The fastest convergence is obtained with a multigrid preconditionner (SA-AMG).

# Memory used

In [ ]:
a = plot.Graph("Max RAM per core used during calculation:","",1,1,[10,5])
for case,label,syntax in cases:
    cols = plot.loadText(case+"/%s.ram" % case)
    a.add(cols[0],cols[1],label="%s" % label)
a.label("Time [s]","RAM [MB]")

# CPU time evolution

In [ ]:
a = plot.Graph("CPU time of pressure solve during calculation:","",1,1,[10,5])
for case,label,syntax in cases:
    cols = plot.loadText(case+"/%s.cpu" % case)
    a.add(cols[0],cols[1],label="%s" % label)
    cols = plot.loadText(case+"/PAR_%s.cpu" % case)
    a.add(cols[0],cols[1],label="%s (4 MPI cores)" % label)
a.label("Time step","CPU [s]")
a.scale(yscale='log')
#a.subplot.set_xticks(range(1,6))

The faster solver is BICGSTAB with SA-AMG preconditionner.

### Computer performance:

In [ ]:
run.tablePerf()

# Conclusions

XXX